In [1]:
import dlt
import requests
from datetime import datetime
import os

In [2]:
os.environ['DESTINATION__CLICKHOUSE__CREDENTIALS__HOST'] = '192.168.1.66:9000'
os.environ['DESTINATION__CLICKHOUSE__CREDENTIALS__DATABASE'] = 'metrics'
os.environ['DESTINATION__CLICKHOUSE__CREDENTIALS__USER'] = 'root'
os.environ['DESTINATION__CLICKHOUSE__CREDENTIALS__PASSWORD'] = '020491'

In [3]:
# Настройки
url = 'https://api.appmetrica.yandex.ru/logs/v1/export/events.json'
oauth_token = 'y0_AgAAAAAisvRKAAutVAAAAAEC7JiLAAAtHPjoix1FhahM1KuvZy5qiQ3ZvQ'
headers = {'Authorization': f'OAuth {oauth_token}'}
application_id = '3757420'
fields = 'event_datetime,event_name,event_json,appmetrica_device_id,session_id'  # Укажите нужные поля

In [6]:
date_since = datetime(2023, 12, 5)  # Начальная дата
date_until = datetime(2023, 12, 10)  # Конечная дата

start_date = date_since
while start_date <= date_until:
    fetch_data_for_day(start_date)
    start_date += timedelta(days=1)

: 

In [4]:
pipeline = dlt.pipeline(
    pipeline_name="app_to_clickhouse",  # Имя конвейера
    destination="clickhouse",  # Целевое хранилище (ClickHouse)
    dataset_name="metrica_events"  # Имя набора данных
    )

In [5]:
# Функция для получения данных за один день
def fetch_data_for_day(date):
    params = {
        'application_id': application_id,
        'date_since': date.strftime('%Y-%m-%d'),
        'date_until': date.strftime('%Y-%m-%d'),
        'fields': fields
    }
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 202:
        print(f"Ожидаю 200 для даты {date}...")
        while response.status_code == 202:
            print(response.status_code)
            time.sleep(15)
            response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        print(f"Загружаю данные за {date}...")
        data = response.json()
        
        batch_size = 1000  # Размер пакета
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            pipeline.run(batch, table_name="metrica_events", write_disposition="append")
        del response
        del data
        gc.collect()

    else:
        print(f"Ошибка для даты {date}: {response.status_code}")